Com este dataset vamos avaliar preços de casas em Melbourne. Os valores estã expressos em Dolar Australiano

In [2]:
# importando as bibliotecas que serão usadas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error

In [3]:
# Importando o dataset
df = pd.read_csv('Melbourne_housing_FULL.csv')
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


Tratamento de dados

1. Removendo colunas que não são necessárias

In [4]:
del df['Address']
del df['Method']
del df['SellerG']
del df['Date']
del df['Postcode']
del df['Lattitude']
del df['Longtitude']
del df['Regionname']
del df['Propertycount']

In [5]:
# verificando se há valores ausentes
df.isna().sum()

Suburb              0
Rooms               0
Type                0
Price            7610
Distance            1
Bedroom2         8217
Bathroom         8226
Car              8728
Landsize        11810
BuildingArea    21115
YearBuilt       19306
CouncilArea         3
dtype: int64

2. Removendo os valores ausentes. Como é um exercício vamos eliminar os valores ausentes para ser mais simples

In [6]:
df.dropna(axis=0, how='any', subset=None, inplace=True)

In [7]:
df.isna().sum()

Suburb          0
Rooms           0
Type            0
Price           0
Distance        0
Bedroom2        0
Bathroom        0
Car             0
Landsize        0
BuildingArea    0
YearBuilt       0
CouncilArea     0
dtype: int64

3. Agora vamos converter as conlunas que contêm valores não numéricos usando a técnica 'one-hot-enconding'

In [8]:
df = pd.get_dummies(data=df, columns=['Suburb', 'CouncilArea', 'Type'])

4. Setando as variáveis independentes com X e a variável dependente (Price) como y

In [9]:
X = df.drop('Price', axis=1)
y = df['Price']

Separando os dados em dados de treino e de teste

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = True)

Vamos selecionar o algoritmo e configurar os hiper-parâmetros

In [11]:
model = ensemble.GradientBoostingRegressor(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=30,
    min_samples_split=4,
    min_samples_leaf=6,
    max_features=0.6,
   loss='huber'
)

Explicando os hiperparâmetros:
<ul>
<li>n_estimators: número de árvores de decisão</li>
<li>learning_rate: controla a taxa na qual árvores de decisão adicionais influenciam a previsão geral.</li>
<li>max_depth: define o máximo de camadas para cada árvore de decisão</li>
<li>min_samples_split: o número mínimo requerido de amostras para executar uma nova divisão binária, para criar um novo ramo(branch)</li>
<li>min_samples_leaf: número mínimo de amostras que deve aparecer em cada folha antes de um novo ramo ser implementado. Isso ajuda a atenuar o impacto de outliers e anomalias</li>
<li>max_features: número total de características apresentadas ao modelo ao determinar a melhor divisão</li>
<li>loss: calcula a taxa de erro do modelo. No caso, huber protege contra outliers e anomailias.</li>
</ul>

In [12]:
# Ajustando o modelo
model.fit(X_train, y_train)

GradientBoostingRegressor(loss='huber', max_depth=30, max_features=0.6,
                          min_samples_leaf=6, min_samples_split=4,
                          n_estimators=150)

Avaliando os Resultados

Vamos usar a função predict() para executar o modelo nos dados de treino e avaliar sua performance contra os dados reais (y_train). Faremos isso usando o erro médio absoluto.

In [15]:
mae_train = mean_absolute_error(y_train, model.predict(X_train))
print(f'Erro médio absoluto dos dados de treinamento: {mae_train:.2f}')

Erro médio absoluto dos dados de treinamento: 28081.11


In [17]:
# Usando o mesmo método anterior para os dados de teste
mae_test = mean_absolute_error(y_test, model.predict(X_test))
print(f'Erro médio absoluto dos dados de teste: {mae_test:.2f}')

Erro médio absoluto dos dados de teste: 168490.65


Resultados: <br>1. nosso modelo de treinamento foi eficaz em prever o real valor das propriedades contidas nos dados de treino.<br>Pode parecer que $28081.11 seja uma grande quantia mas esse erro médio é baixo considerando que o range máximo do nosso conjunto de dados é $8 milhões.<br>2. A alta discrepância entre os erros de treinamento e de teste indica que há um overffiting. No nosso caso, esta diferença é exacerbada porque configuramos o modelo para fazer um overffit nos dados de treino, quando configuramos o max_depht = 30

OTIMIZAÇÃO DO MODELO

O primeiro ponto é modificar os hiperparâmetros.<br>Vamos começar ajustando max_depth = 5 e deixar os outros como estão.

In [21]:
model = ensemble.GradientBoostingRegressor(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=5,
    min_samples_split=4,
    min_samples_leaf=6,
    max_features=0.6,
    loss='huber'
)

In [22]:
model.fit(X_train, y_train)

GradientBoostingRegressor(loss='huber', max_depth=5, max_features=0.6,
                          min_samples_leaf=6, min_samples_split=4,
                          n_estimators=150)

In [23]:
# Erro médio absoluto dos dados de treinamento
mae_train = mean_absolute_error(y_train, model.predict(X_train))
print(f'Erro médio absoluto dos dados de treinamento: {mae_train:.2f}')


Erro médio absoluto dos dados de treinamento: 135904.92


Vamos otimizar o modelo agora adicionando mais árvores

In [24]:
model = ensemble.GradientBoostingRegressor(
    n_estimators=250,
    learning_rate=0.1,
    max_depth=5,
    min_samples_split=4,
    min_samples_leaf=6,
    max_features=0.6,
    loss='huber'
)

In [25]:
model.fit(X_train, y_train)

GradientBoostingRegressor(loss='huber', max_depth=5, max_features=0.6,
                          min_samples_leaf=6, min_samples_split=4,
                          n_estimators=250)

In [26]:
# Erros médios absolutos de treino e teste
mae_train = mean_absolute_error(y_train, model.predict(X_train))
print(f'Erro médio absoluto dos dados de treinamento: {mae_train:.2f}')

mae_test = mean_absolute_error(y_test, model.predict(X_test))
print(f'Erro médio absoluto dos dados de teste: {mae_test:.2f}')

Erro médio absoluto dos dados de treinamento: 123086.11
Erro médio absoluto dos dados de teste: 163098.00


Esta segunda otimização reduziu a taxa do erro médio absoluto de treino em aproximadamente $11,000 e há uma pequena lacuna entre os resultados de treino e teste.<br>Essas duas otimizações servem para mostrar a importância de se compreender individualmente os hiperparâmetros.